# GradientClimber Demo

This notebook is adapted for use in the Gradient Climber application using the Garden Buddy application as an example.  We use the Gradient Climber Quality Attribute Scenarios as guidence for the required Properties and Conditions.

NOTE: this demo has an additional set of requirements than MLTE. You can install them with the command: 

`poetry install --with demo`


## 1.1 Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. For details about setting the context, see the ``session.py`` module.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

## 1.2 Build/import a `NegotiationCard`

In MLTE, we negotiation requirements with the help of a `NegotiationCard`. This can be done manually through code, but it is easier to use the MLTE UI to do so. Below we are copying a pre-built one that applies to this scenario. In MLTE, we define requirements by constructing a `NegotiationCard` that will include explicit Quality Attribute Scenarios with the requirements.

In [ ]:
%%bash
(cd ../ && cp -r ./sample_store/* ./store/)

In [ ]:
from mlte.negotiation.artifact import NegotiationCard
card = NegotiationCard.load()

The examples below relate to a reinforcement learning system based on the mountain car example [Andrew W. Moore (1990). Efficient Memory-Based Learning for Robot Control (PhD thesis, University of Cambridge)].

In [ ]:
# Print basic system information in the negogation card to give context to the Quality attribute scenarios
print("The goals of the system are: ")
for goal in card.system.goals:
    print("  -", goal.description)
    for metric in goal.metrics:
        print("\t as measured by: ", metric)
print("The data used/to be used in training the system is:")
for data in card.data:
    print("  -", data.description, " for ", data.purpose, " from ", data.source)

print(
    "And the usuage context for the model is that ", card.system.usage_context
)

if len(card.system.risks) > 0:
    print(
        "The stakeholder perceived risks to the system are: ",
        "\n  -".join(card.system.risks),
    )

The following are the QASs that we want to validate through the use of MLTE. 

In [ ]:
# Print all quality attribute scenarios
card.print_quality_scenarios()

## 1.3 Define a TestSuite

In the first phase of SDMT, we define a `TestSuite` that represents the tests the completed model must will have to pass in order to be acceptable for use in the system into which it will be integrated.

In MLTE, we define the tests that will be required for the different requirements in a `TestSuite`. Note that a new `Evidence` types (`MultipleRanksums`) had to be created in this case to handle the data and `Validator` for that case, and two stand-alone `Validator`s were defined in `validators.py` to validate data using existing `Evidence` types.

First we need to load up our `NegotiationCard`, so we can get the list of ids of its quaity attribute scenarios, that will be added to the `TestCase`s here. Those ids are the way to link the `TestCase`s to their quality attribute requirements.

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
card.save(force=True)
card.print_quality_scenarios()

Now we can create our `TestSuite`, consisting of a list of `TestCases`, each of them addressing one or more Quality Attribute Scenarios from our `NegotiationCard`. When defining the `TestCase`s below, we need to set the id of the corresponding Quality Attribute Scenario we want to test in its "quality_scenarios" attribute.

In [ ]:
from mlte.tests.test_case import TestCase
from mlte.tests.test_suite import TestSuite

# The Evidence types we will use to validate each condition.
from utils import validators
from mlte.validation.validator import Validator

test_suite = TestSuite(
    test_cases=[
        # Functional correctness
        TestCase(
            identifier="functional correctness",
            goal="Ensure that the model navigates vehicle to goal location within required time",
            quality_scenarios=["default.card-qas_001"],
            validator=validators.passed_percent_more_or_equal_then(0.9),
        ),
        # Reliability
        TestCase(
            identifier="reliability",
            goal="Ensure model outputs an action that improves expected reward 99.9% of the time",
            quality_scenarios=["default.card-qas_002"],
            validator=validators.passed_percent_more_or_equal_then(0.999),
        ),
        # Compliance (position)
        TestCase(
            identifier="gradient climber position accuracy",
            goal="Check if model complies with position requirements",
            quality_scenarios=["default.card-qas_003"],
            validator=validators.passed_percent_more_or_equal_then(1.0),
        ),
        # Compliance (velocity)
        TestCase(
            identifier="gradient climber velocity accuracy",
            goal="Check if model complies with position requirements",
            quality_scenarios=["default.card-qas_004"],
            validator=validators.passed_percent_more_or_equal_then(1.0),
        ),
        # Deployability
        TestCase(
            identifier="deployability",
            goal="Verified that the q-table file is available and of a manageable size.",
            quality_scenarios=["default.card-qas_006"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
        # Portability
        TestCase(
            identifier="portability",
            goal="Data is in the form of an npy file which is portable by design.",
            quality_scenarios=["default.card-qas_007"],
            validator=Validator.build_info_validator(
                "Inspect project code and documentation."
            ),
        ),
    ]
)
# The full test suite.

test_suite.save(parents=True, force=True)